In [1]:
from tda.experiments.ocsvm_detector.ocsvm_detector_binary import Config, get_all_embeddings
from tda.embeddings import EmbeddingType, KernelType
from tda.models.architectures import mnist_lenet

2020-01-14 20:11:07,754 - Devices - INFO - Found 0 compatible with CUDA


In [2]:
config = Config(
    embedding_type=EmbeddingType.PersistentDiagram,
    kernel_type=KernelType.SlicedWasserstein,
    thresholds='0.9',
    epochs=50,
    dataset="MNIST",
    architecture=mnist_lenet.name,
    train_noise=0.0,
    dataset_size=10,
    successful_adv=1,
    attack_type="FGSM",
    identical_train_samples=1,
    noise=0.0,
    
    num_iter=1,
    height=1,
    hash_size=1,
    node_labels=0,
    steps=1
)

In [3]:
embedding_train, embedding_test, adv_embeddings, thresholds, stats, stats_inf = get_all_embeddings(config)

2020-01-14 20:11:18,577 - Datasets - INFO - Instantiated dataset MNIST with validation_size 1000
2020-01-14 20:11:18,578 - Models - INFO - Filename = /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../trained_models/mnist_mnist_lenet_50_epochs.model 

/home/elvis/anaconda3/envs/tda/lib/python3.6/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'tda.models.architectures.Architecture' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
2020-01-14 20:11:18,591 - Models - INFO - Loaded successfully model from /home/elvis/CODE/FORKED/TDA_for_adv_robustness/tda/../trained_models/mnist_mnist_lenet_50_epochs.model
2020-01-14 20:11:18,592 - Thresholds - INFO - Detected uniform threshold
2020-01-14 20:11:18,593 - Thresholds - INFO - My received thresholds {(-1, 0): 0.9, (0, 1): 0.9,

TypeError: get_graph_dataset() got an unexpected keyword argument 'source_dataset_name'

In [ ]:
logger.info(f"Using kernel {config.kernel_type} with embeddings {config.embedding_type}")

if config.kernel_type == KernelType.RBF:
    param_space = [
        {'gamma': gamma}
        for gamma in np.logspace(-6, -3, 10)
    ]
elif config.kernel_type == KernelType.SlicedWasserstein:
    param_space = [
        {'M': 20, 'sigma': 5 * 10 ** (-1)},
    ]
else:
    raise NotImplementedError(f"Unknown kernel {config.kernel_type}")

gram_train_matrices = {i: get_gram_matrix(
    kernel_type=config.kernel_type,
    embeddings_in=embedding_train,
    embeddings_out=None,
    params=param
)
    for i, param in enumerate(param_space)
}
logger.info(f"Computed all Gram train matrices !")

In [ ]:
all_results = {
        epsilon: evaluate_embeddings(
            gram_train_matrices=gram_train_matrices,
            embeddings_train=embedding_train,
            embeddings_test=embedding_test,
            adv_embeddings=adv_embeddings[epsilon],
            param_space=param_space,
            kernel_type=config.kernel_type
        )
        for epsilon in adv_embeddings
    }

logger.info(all_results)